Análisis de Precios de Viviendas en California: El objetivo es identificar los factores que influyen en los altos precios de las viviendas en California. Usarás un conjunto de datos con información sobre diversas propiedades.

La pregunta
Queremos predecir manualmente para un inversor qué zona es la mejor para invertir en bienes raíces en California
Puede haber muchos enfoques para ver esto. Vamos a ver uno de ellos. Es bastante básico

Loading the libreries

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Pandas: se utiliza para la manipulación de datos

Matplotlib: se utiliza para la visualización estática personalizable

Seaborns: se utiliza para la visualización de datos


Loading the dataset

In [18]:
#carga de datos
data = pd.read_csv("../datos/housing.csv")




Datos cargados exitosamente!


Ckecking the dataset

En este paso vamos a conocer nuestro conjunto de datos.

Checking the head


Verifiquemos la columna y los primeros 5 valores en cada columna

In [33]:
data.head()

NameError: name 'data' is not defined

Checking the description

Deberíamos conocer la media, la mediana y otros valores de distribución estadística del conjunto de datos. Vamos a comprobarlos.

In [34]:
data.describe()

NameError: name 'data' is not defined

Checking Data types

Veamos los tipos de datos de las columnas.

In [35]:
data.dtypes

NameError: name 'data' is not defined

Aquí vemos que cada columna es un número de tipo float 64